In [1]:
from keras import backend as K
from keras.models import load_model
from keras.preprocessing import image
from keras.optimizers import Adam
from imageio import imread
import numpy as np
from matplotlib import pyplot as plt

from models.keras_ssd512 import ssd_512
from keras_loss_function.keras_ssd_loss import SSDLoss
from keras_layers.keras_layer_AnchorBoxes import AnchorBoxes
from keras_layers.keras_layer_DecodeDetections import DecodeDetections
from keras_layers.keras_layer_DecodeDetectionsFast import DecodeDetectionsFast
from keras_layers.keras_layer_L2Normalization import L2Normalization

from ssd_encoder_decoder.ssd_output_decoder import decode_detections, decode_detections_fast

from data_generator.object_detection_2d_data_generator import DataGenerator
from data_generator.object_detection_2d_photometric_ops import ConvertTo3Channels
from data_generator.object_detection_2d_geometric_ops import Resize
from data_generator.object_detection_2d_misc_utils import apply_inverse_transforms

%matplotlib inline

Using TensorFlow backend.
c:\users\kleck\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\kleck\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\kleck\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16

In [2]:
img_height = 512
img_width = 512
n_classes = 1

K.clear_session()

model = ssd_512(image_size=(img_height, img_width, 3),
                n_classes=1,
                mode='inference',
                l2_regularization=0.0005,
                scales=[0.07, 0.15, 0.3, 0.45, 0.6, 0.75, 0.9, 1.05], #scales originalni pro SSD
                aspect_ratios_per_layer=[[1.0, 2.0, 0.5],
                                         [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
                                         [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
                                         [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
                                         [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
                                         [1.0, 2.0, 0.5],
                                         [1.0, 2.0, 0.5]],
               two_boxes_for_ar1=True,
               steps=[8, 16, 32, 64, 128, 256, 512],
               offsets=[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5],
               clip_boxes=False,
               variances=[0.1, 0.1, 0.2, 0.2],
               normalize_coords=True,
               subtract_mean=[123, 117, 104],
               swap_channels=[2, 1, 0],
               confidence_thresh=0.185,
               iou_threshold=0.5,
               #confidence_thresh=0.2,
               #iou_threshold=0.2,
               top_k=200,
               nms_max_output_size=400)



#nacteni modelu
weights_path = 'load_models/ssd512_person_detection.h5'

model.load_weights(weights_path, by_name=True)

adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0) #adam optimaizer

ssd_loss = SSDLoss(neg_pos_ratio=3, alpha=1.0)

model.compile(optimizer=adam, loss=ssd_loss.compute_loss)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.


In [3]:
n_classes=1 #pocet rid

#Caltech_images_dir_test= 'E:/ssd/dataset_fin3/test/Images'
my_images_dir_test='E:/ssd/testMyData/Images'

#Caltech_annotations_test ='E:/ssd/dataset_fin3/test/test.json'
my_annotations_test='E:/ssd/testMyData/myTest.json'

classes = ['person']
dataset = DataGenerator()
dataset.parse_json(images_dirs=[my_images_dir_test],
                         annotations_filenames=[my_annotations_test],
                         ground_truth_available=True,
                         include_classes='all',
                         verbose = False,
                         ret=False)



convert_to_3_channels = ConvertTo3Channels()
resize = Resize(height=img_height, width=img_width)
batch_size=1
generator = dataset.generate(batch_size=1,
                             shuffle=True,
                             transformations=[convert_to_3_channels,
                                              resize],
                             returns={'processed_images',
                                      'filenames',
                                      'inverse_transform',
                                      'original_images',
                                      'original_labels'},
                             keep_images_without_gt=False)



In [4]:
import numpy as np
from sklearn.metrics import average_precision_score


confidence_threshold = 0.185
iou_th=0.5


tru=np.array(1)
score=np.array(1)
TP = 0 #true positive
all_ground = 0 
all_det = 0 

Precision = 0
Recall = 0
Average_P = 0


for k in range(0, dataset.get_dataset_size()):
#for k in range(0, 1):

    batch_images, batch_filenames, batch_inverse_transforms, batch_original_images, batch_original_labels = next(generator)
    '''
    print("Image:", batch_filenames[0])
    print()
    print("Ground truth boxes:\n")
    print(np.array(batch_original_labels[0]))
    '''
    # Predikce na zaklade modelu
    y_pred = model.predict(batch_images)
    y_pred_thresh = [y_pred[k][y_pred[k,:,1] > confidence_threshold] for k in range(y_pred.shape[0])]
    y_pred_thresh_inv = apply_inverse_transforms(y_pred_thresh, batch_inverse_transforms)

    np.set_printoptions(precision=2, suppress=True, linewidth=90)
    #print("Predicted boxes:\n")
    #print('   class   conf xmin   ymin   xmax   ymax')
    #print(y_pred_thresh_inv[i])


    #IoU_test=0.5
    #Vypocet iou + porovnani
    
    h=len(batch_original_labels[0])
    all_ground=all_ground+h
    all_det=all_det+len(y_pred_thresh_inv[0])
    for box2 in y_pred_thresh_inv[0]:
        xmin2 = box2[2]
        ymin2 = box2[3]
        xmax2 = box2[4]
        ymax2 = box2[5]


        Precision_decision=True
        
        z=0
        for box1 in batch_original_labels[0]:
            xmin1 = box1[1]
            ymin1 = box1[2]
            xmax1 = box1[3]
            ymax1 = box1[4]


            xmin = max(xmin1,xmin2)
            ymin = max(ymin1,ymin2)
            xmax = min(xmax1,xmax2)
            ymax = min(ymax1,ymax2)
            interArea = abs(max((xmax - xmin, 0)) * max((ymax - ymin), 0))

            # Vypocet area predikovanych ohraniceni a originalnich ohr.
            boxAArea = abs((xmax1 - xmin1) * (ymax1 - ymin1))
            boxBArea = abs((xmax2 - xmin2) * (ymax2 - ymin2))

            #iou
            iou = interArea / float(boxAArea + boxBArea - interArea)
            
            if iou>=iou_th:
                TP=TP+1
                Precision_decision=False
                tru=np.append(tru,1)
                score=np.append(score,box2[1])
                break
                
        if Precision_decision==True:
            tru=np.append(tru,0)
            score=np.append(score,box2[1])


Precision=(TP / all_det)
Recall=(TP / all_ground)
if Recall>1:
    Recall=1
Average_P=(average_precision_score(tru, score))
F1=2 * (Precision * Recall) / (Precision + Recall)


In [5]:
print('Práh modelu: ',confidence_threshold)
print('Precision: ',Precision)
print('Recall: ',Recall)
print('AP: ',Average_P)
print('F1: ',F1)

Práh modelu:  0.185
Precision:  0.7692307692307693
Recall:  0.9876543209876543
AP:  0.9743149926243251
F1:  0.864864864864865
